In [1]:
import sys
sys.path.append("../..")
from thesis.utils.utils import setup_ray, save, load, Experiment
path = "D:/Master/Masterarbeit/thesis"
setup_ray(path = path, unidirectional = False, seed=69)

2023-04-03 07:09:47,461	INFO worker.py:1538 -- Started a local Ray instance.


In [2]:
env_args = dict(
    fleetsize = 16,
    max_fleetsize = 20,    
    pseudo_routing = False,
    pseudo_dispatcher = False,
    pseudo_dispatcher_clever = False,
    #pseudo_dispatcher_distance = 0.3,
    routing_agent_death= True,
    death_on_target = False,
    transform_dispatching_partobs = True,
    direction_reward = -0.1,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 10,
        #reward_reached_target_by_time = True, 
        reward_wrong_target = -1,
        reward_removed_for_block = -5, 
        reward_target_distance = 0,
        reward_invalid= -0.1,
        reward_duration = -0.5,
        block_timeout = 120,
        station_separate = False,
        reward_accepted_in_station = 2,
        reward_declined_in_station = -1,
        #reward_part_completed = 5,
        #reward_geo_operation=1,
        #reward_rework_operation=1,
        #reward_respot_operation=1,
        reward_reduce = -0.02,
        reward_balance = -5,
        routing_interval = 2,
        dispatching_interval=30,
        io_quote = 0.9  ,
        availability = 0.9,
        mttr = 5,
        fixed_fleets = True,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "gnn_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            with_agvs=True,
            with_stations = False,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
            env_type = "matrix",
            n_convolutions = 2
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=32,
            with_action_mask=True,
            with_agvs=True,
            with_stations = True,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
            #with_main_pos_noise = True,
        )
    )
)

In [4]:
exp = Experiment("matrix_together")
for seed in [42]:
    exp.experiment(
        path = path,
        env_args = env_args, 
        agv_model = agv_model,
        dispatcher_model=dispatcher_model,
        run_name="01_two_fleets", 
        env = "matrix",
        algo = "double",
        n_intervals =0,
        backup_interval=50,
        batch_size=400, 
        seed = seed,
        algo_params = {"gamma":0.9, "exploration_config":{"warmup_timesteps": 0,"epsilon_timesteps": 200000,"final_epsilon": 0.02,"initial_epsilon": 0.02,"type": "EpsilonGreedy"}},
        lr = 1e-4,
        two_fleets = True,
        load_agv="../../models/matrix_together/01_two_fleets_8_20_2023-04-02_22-17-05/checkpoint_000400"
    )
    #load(exp.trainer, "agv", "../../models/trained_routing_new")
    #load(exp.trainer, "dispatcher1", "../../models/trained_disp_1")
    #load(exp.trainer, "dispatcher2", "../../models/trained_disp_2")
    exp.keep_training(8)

2023-04-03 07:09:58,965	INFO algorithm.py:501 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_29859_h7abp45p)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
2023-04-03 07:10:27,397	INFO trainable.py:172 -- Trainable.setup took 28.440 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-04-03 07:10:28,173	INFO trainable.py:790 -- Restored on 127.0.0.1 from checkpoint: ..\..\models\matrix_together\01_two_fleets_8_20_2023-04-02_22-17-05\checkpoint_000400
2023-04-03 07:10:28,175	INFO trainable.py:799 -- Current state after restoring: {'_iteration': 400, '_timesteps_total': None, '_time_total': 20762.562623262405, '_episodes_total': 182}


In [ ]:
#exp.trainer.load_checkpoint("../../models/matrix_together/01_two_fleets_8_20_2023-04-02_22-17-05/checkpoint_000400")

In [ ]:
#save(exp.trainer, "agv", "../../models/trained_together_agv")

In [ ]:
#save(exp.trainer, "dispatcher1", "../../models/trained_together_dispatcher1")
#save(exp.trainer, "dispatcher2", "../../models/trained_together_dispatcher2")